In [ ]:
!pip install pysrt requests sentencepiece transformers==4.33 sacremoses  sacrebleu

In [ ]:
import requests
import pysrt
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch


In [ ]:
model_name = "zaanind/nllb-ensi-v1-tuning-subs" #"facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
def dotranslate(text):
  inputs = tokenizer(text, return_tensors="pt")
  translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["sin_Sinh"])
  out = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
  return out


In [ ]:
from google.colab import files

print("<< Upload Subtitle file to translate >>")
uploaded = files.upload()

for fn in uploaded.keys():
  filenm = fn
  print(fn)


print("<<Translating subtitle in progress>>")

# Read the subtitles from a file using pysrt
subtitles = pysrt.open(filenm)

# Iterate through each subtitle line and translate
for subtitle in subtitles:
    input_text = subtitle.text

    translated_text = dotranslate(input_text)
    print(translated_text)

    # Update the subtitle with the translated text
    subtitle.text = translated_text

# Save the modified subtitles to a new file
subtitles.save('translated_subtitle_file.srt')

print("<<downloading.. if not, click>>")

files.download('translated_subtitle_file.srt')
